# Road Following - Build TensorRT model for live demo

In this notebook, we will optimize the model we trained using TensorRT.

## Load the trained model

1. This script is only for the models trained with "train_model_PC.ipynb". If the model is trained with "train_model.ipynb", you should use live_demo_build_trt_o.ipynb.
2. Before executing this script, you should copy the whole directory and the trained model files saved inside it to "Home/model_repo", where HOME is the home directory of your Jetson Nano (here, we assume HOME=/home/cuterbot). The trained model files is saved in the directory variable "DIR_RC_MODEL_REPO" you set in "train_model_PC.ipynb", which should be in the PC used for training.
3. The pytorch model will be uploaded after you selected the model from the selection widget in the cell below.
> Note: Please make sure the file has been uploaded fully from the log console of jupyter lab.

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [1]:
import torchvision
import torch
from jetbot.utils import model_selection
import os 
import ipywidgets.widgets as widgets
from ipywidgets.widgets import Box, HBox, VBox, Layout, Label, Output
import traitlets

# The path of trt models is 'MODEL_REPO_DIR_DOCKER' which is set in /jetbot/utils/model_selection.py,
# which may be modified if you change the file path of trt models, 'MODEL_REPO_DIR_DOCKER' or dir_model_repo.

dir_model_repo = os.environ['MODEL_REPO_DIR_DOCKER']
print(dir_model_repo)

# pth_ms = model_selection(core_library = "Pytorch")  # if 'MODEL_REPO_DIR_DOCKER' is used.
pth_ms = model_selection(core_library = "Pytorch", dir_model_repo=dir_model_repo)

/workspace/model_repo


In [2]:
pth_ms.model_function = "classifier"

model_type_widget = widgets.Select(options=pth_ms.model_type_list, value=pth_ms.model_type_list[0],
                                      description='Model Type:')
traitlets.dlink((pth_ms, 'model_type_list'), (model_type_widget, 'options'))
traitlets.dlink((model_type_widget, 'value'), (pth_ms, 'model_type'))

model_path_widget = widgets.Select(options=pth_ms.model_path_list, description='Model Path:',
                                      layout=Layout(width='65%'))
traitlets.dlink((pth_ms, 'model_path_list'), (model_path_widget, 'options'))
traitlets.dlink((model_path_widget, 'value'), (pth_ms, 'model_path'))

path_preprocess = pth_ms.preprocess_path  # the preprocess for torch model is used for trt model
# print('preprocess path:', path_preprocess)

Next, load the pytoch model and the trained weights from the model_path (e.g.``best_steering_model_xy_<<pth_model_name>>.pth``) file that you uploaded.

In [3]:
out = Output(layout={'border': '2px solid black'})

In [4]:
device = torch.device('cuda')

@out.capture()
def load_trained_model(model_path, pth_model_name):
    from jetbot.utils.model_selection import load_tune_pth_model

    print("start load trained model -- \n model name : ", pth_model_name, '\n model path: ', model_path)

    model, model_type, preprocess_wrap = load_tune_pth_model(pth_model_name=pth_model_name, pretrained=False)  
    model.load_state_dict(torch.load(model_path))

    # Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device. 
    model.to(device).eval().half()
    # model = model.cuda().eval().half()

    return model_type, model, preprocess_wrap

## TensorRT conversion:

1. If you are running with docker container, you may not need to do the following installation.
> Note: If your setup does not have `torch2trt` installed, you need to first install `torch2trt` by executing the following in the console.
```bash
    cd $HOME
    git clone https://github.com/NVIDIA-AI-IOT/torch2trt
    cd torch2trt
    sudo python3 setup.py install
```
> Convert and optimize the model using torch2trt for faster inference with TensorRT. Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.
> This optimization process can take a couple minutes to complete.

2. After finishing TensorRT engine conversion, the created TensorRT model engines will be stored in same directory you specified to store the step 2 above ("Home/model_repo/road_following"), and the mata data file 'trt_model_tbl.csv' will be updated in directory "Home/model_repo".
3. Then, you can excute the "live_demo_light_trt.ipynb" to simulate the road following function with the converted TensorRT engine.

In [5]:
@out.capture()
def trt_conversion(pth_model_name, model, preprocess):
    from torch2trt import torch2trt
    import tensorrt as trt # Logger : ERROR, INFO, VERBOSE, WARNING
    
    print("start building TRT model -- ")

    preprocess.to(device).eval().half()
    data = torch.zeros((1, 3, 224, 224)).to(device).half()
    data = preprocess(data).to(device).half()
    # data = preprocess(data).cuda().half()
    
    '''
    if pth_model_name == 'inception_v3':
        data = torch.zeros((1, 3, 299, 299)).cuda().half()   # inception_v3
    else:
        data = torch.zeros((1, 3, 224, 224)).cuda().half()  # resnet
    '''   
    model_trt = torch2trt(model, [data], fp16_mode=True, log_level=trt.Logger.VERBOSE)
    
    return model_trt

### Save the optimized model using the cell below

In [6]:
# Save the optimized model using the cell below
@out.capture()
def save_trt_model(pth_model_name, model_type, model_trt, path_trt_model, path_trt_model_preprocess):
    import pandas as pd
    print("saving built trt model  --")
    
    torch.save(model_trt.state_dict(), path_trt_model)

    df_file = os.path.join(dir_model_repo, 'trt_model_tbl.csv')
    if os.path.isfile(df_file):
        df = pd.read_csv(df_file, header=None)
    else:
        df = pd.DataFrame()

    path_trt_model_tbl = path_trt_model.replace(dir_model_repo, '.')
    path_trt_model_preprocess_tbl = path_trt_model_preprocess.replace(dir_model_repo, '.')
    df = df.append([["classifier", model_type, path_trt_model_tbl, path_trt_model_preprocess_tbl]], ignore_index = False)
    df = df.drop_duplicates()
    df.to_csv(df_file, header=False, index=False)
    return

In [7]:
@out.capture()
def start_trt_conversion(change):
    from jetbot.utils import tv_classifier_preprocess
    button_OK.disabled=True
    
    all_model = True     # set True to convert all pytorch models
    re_convert = True  # wether to do trt conversion of conerted trt model
    
    list_model_path = []
    if not all_model:
        list_model_path.append([model_path_widget.value, pth_ms.preprocess_path])
    else:
        import pandas as pd
        from jetbot.utils.model_selection import HEAD_LIST
        df = pd.read_csv(os.path.join(dir_model_repo, "torch_model_tbl.csv"), header=None, names=HEAD_LIST) # load torch models list
        mf = df[df.model_function == "classifier"]  # select all classifier models
        for mp in mf.loc[:, ['model_path', 'preprocess_path']].values.tolist():
            list_model_path.append([os.path.join(dir_model_repo, mp[0].split('/', 1)[-1]), 
                                    os.path.join(dir_model_repo, mp[1].split('/', 1)[-1])])
        
    for model_path in list_model_path:
        torch_model_path = model_path[0]
        pth_model_name = model_path[0].split('/')[-1].split('.')[0].split('_', 4)[-1]
        path_trt_model = model_path[0].replace('.pth', '_trt.pth')
        path_trt_model_preprocess = model_path[1]
        print("path_trt_model: %s \n" % path_trt_model, "path_trt_model_preprocess: %s" % path_trt_model_preprocess)

        if os.path.isfile(path_trt_model) and not re_convert:
            pass
        else:
            model_type, model, preprocess_wrap = load_trained_model(torch_model_path, pth_model_name)
            if preprocess_wrap is None:  # load pre-stored preprocess module of the trained model
                preprocess = tv_classifier_preprocess()
                preprocess.load_state_dict(torch.load(path_trt_model_preprocess))
            else:  # used the preprocess from load_tune_pth_model
                preprocess = preprocess_wrap[0]            
            model_trt = trt_conversion(pth_model_name, model, preprocess)
            save_trt_model(pth_model_name, model_type, model_trt, path_trt_model, path_trt_model_preprocess)
            print("Building finished, and TRT model is saved in -- %s \n" % path_trt_model)
    print("TRT model Conversion completed!")    
    button_OK.disabled=False
    
display(HBox([model_type_widget, model_path_widget]))

button_OK = widgets.Button(description='OK', tooltip='Click to start', icon="start")
button_OK.style.button_color='lightblue'
button_OK.on_click(start_trt_conversion)
display(button_OK)
out

Button(description='OK', icon='start', style=ButtonStyle(button_color='lightblue'), tooltip='Click to start')

Output(layout=Layout(border='2px solid black'))

## Next
Open live_demo_trt.ipynb to move JetBot with the TensorRT optimized model.